In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_file_location = "../data/raw/steam-200k.csv"
processed_directory = "../data/processed"
header_names = ['user_id', 'game_name', 'purchase-play', 'indicator-hours', 'empty']
data = pd.read_csv(raw_file_location, names=header_names)

In [3]:
data.head()

,user_id,game_name,purchase-play,indicator-hours,empty
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [4]:
"""
Is purchase always associated with a 1.0?
The answer is yes. 
"""
#subset data into a purchase dataset and a play dataset based on purchase-play
purchase = data.loc[data['purchase-play'] == 'purchase']
play = data.loc[data['purchase-play'] == 'play']

In [5]:
print("The the average of purchase is: ", np.mean(purchase['indicator-hours']))
purchase.head()

('The the average of purchase is: ', 1.0)


,user_id,game_name,purchase-play,indicator-hours,empty
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
2,151603712,Fallout 4,purchase,1.0,0
4,151603712,Spore,purchase,1.0,0
6,151603712,Fallout New Vegas,purchase,1.0,0
8,151603712,Left 4 Dead 2,purchase,1.0,0


In [6]:
print("The the average hours played is: ", np.mean(play['indicator-hours']))
play.head()

('The the average hours played is: ', 48.878063243911484)


,user_id,game_name,purchase-play,indicator-hours,empty
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
3,151603712,Fallout 4,play,87.0,0
5,151603712,Spore,play,14.9,0
7,151603712,Fallout New Vegas,play,12.1,0
9,151603712,Left 4 Dead 2,play,8.9,0


In [7]:
"""Remove the empty column"""
play = play[['user_id', 'game_name', 'indicator-hours']]
play.head()

,user_id,game_name,indicator-hours
1,151603712,The Elder Scrolls V Skyrim,273.0
3,151603712,Fallout 4,87.0
5,151603712,Spore,14.9
7,151603712,Fallout New Vegas,12.1
9,151603712,Left 4 Dead 2,8.9


In [8]:
"""Save this file as a csv"""
play.to_csv(processed_directory+"/play_tidy.csv")

In [9]:
unique_users = play['user_id'].unique()
unique_games = play['game_name'].unique()

print("The number of unique users are: ", len(unique_users))
print("The number of unique games are: ", len(unique_games))

('The number of unique users are: ', 11350)
('The number of unique games are: ', 3600)


In [ ]:
"""One Hot Encoding the data"""
play_one = pd.DataFrame({'user':unique_users})
zero_array = np.zeros(unique_users.shape)
for i in unique_games: 
    play_one[i] = zero_array

In [ ]:
ticker = 0
for j in unique_users:  
    if j%(len(unique_users)/10)==0: 
        print(str(ticker*10)+"%")
        ticker += 1
        
    temp = play.loc[data['user_id'] == j]
    games = np.array(temp['game_name'])
    hours = np.array(temp['indicator-hours'])
    for k, game in enumerate(games): 
        indices = play_one.loc[play_one['user']==j].index
        play_one[game][indices] = hours[k]
        

/Users/Muf/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [ ]:
"""faster way"""
play_dummies = pd.get_dummies(play['game_name'])

In [ ]:
play_dummies.head()

In [35]:
#data preprocessing

def getCustomerID(user,users):
    return int(users.index(user))

def getGameID(game,games):
    return int(games.index(game))

#create customers list
customers=[]
for cust in unique_users:
    customers.append(cust)
#create customers file
customers_df= pd.DataFrame(customers, columns=['user'])
customers_df.to_csv(processed_directory+"/customers.csv")

#create games list
games=[]
for game in unique_games:
    games.append(game)
#create games file
games_df= pd.DataFrame(games, columns=['game'])
games_df.to_csv(processed_directory+"/games.csv")

In [36]:
#create hours played file
hours_df= pd.DataFrame(columns=['userIndex','gameID','hours'])
for index, row in play.iterrows():
    hours_df = hours_df.append({'userIndex':getCustomerID(row['user_id'],customers),'gameID':getGameID(row['game_name'],games),'hours':row['indicator-hours']}, ignore_index=True)
hours_df.to_csv(processed_directory+"/hours.csv")
print 'done'

done


In [34]:
print hours_df.head()

   userIndex  gameID  hours
0        0.0     0.0  273.0
1        0.0     1.0   87.0
2        0.0     2.0   14.9
3        0.0     3.0   12.1
4        0.0     4.0    8.9


In [ ]:
#create hours play matrix
playM=np.zeros((len(unique_users),len(unique_games)))
playM.fill(np.nan)

for index, row in hours_df.iterrows():
    playM